<a href="https://colab.research.google.com/github/Schimidel/UGRH-s-PrecipitacaoeTemperatura/blob/main/Download_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Parte 1 - separação das UGRHs em um dataframe

Os shapes de cada UGRHs estão disponibilizados no [github](https://github.com/andrebelem/Mudancas-Climaticas), conforme disposto nas instruções das atividades realizadas durante o semestre.

In [ ]:
!wget https://github.com/andrebelem/pythonverse/raw/main/datasets/SNIRH_UGRHs_2020.zip

--2025-06-01 23:50:06--  https://github.com/andrebelem/pythonverse/raw/main/datasets/SNIRH_UGRHs_2020.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/andrebelem/pythonverse/main/datasets/SNIRH_UGRHs_2020.zip [following]
--2025-06-01 23:50:06--  https://media.githubusercontent.com/media/andrebelem/pythonverse/main/datasets/SNIRH_UGRHs_2020.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13113509 (13M) [application/zip]
Saving to: ‘SNIRH_UGRHs_2020.zip’

SNIRH_UGRHs_2020.zi 100%[===================>]  12.51M  --.-KB/s    in 0.07s   

2025-06-01 23:50:07 (190 MB/s) - ‘SNIRH_

In [ ]:
import geopandas as gpd

df_ugrh = gpd.read_file('/content/SNIRH_UGRHs_2020.zip')
df_ugrh

,fid_,NOME,FID_UGRH,geometry
0,0,Bacias Litorâneas Estaduais do CE,60,"POLYGON ((-37.99711 -5.23233, -37.99755 -5.232..."
1,0,Bacias Litorâneas Estaduais da PB,58,"POLYGON ((-35.6821 -7.70278, -35.68224 -7.7029..."
2,0,Bacias Litorâneas Estaduais de SE,55,"POLYGON ((-37.33509 -11.45245, -37.33511 -11.4..."
3,0,Parnaíba,34,"POLYGON ((-40.5147 -7.31446, -40.51525 -7.3140..."
4,0,Bacias Interestaduais do Litoral PB/RN,31,"POLYGON ((-36.26099 -6.84833, -36.26125 -6.846..."
...,...,...,...,...
59,0,Bacias Litorâneas Estaduais de PE,57,"MULTIPOLYGON (((-37.02205 -8.29203, -37.02003 ..."
60,0,Bacias Litorâneas Estaduais do MA,61,"MULTIPOLYGON (((-46.09129 -1.16304, -46.09126 ..."
61,0,Bacias Litorâneas Estaduais do PA,62,"MULTIPOLYGON (((-49.99453 -1.38552, -49.99512 ..."
62,0,Bacias Litorâneas Estaduais do AP,63,"MULTIPOLYGON (((-51.60389 4.16922, -51.6061 4...."


In [ ]:
%%capture

!pip install -U xarray[complete] # reinstala todo o xarray (útil nessa versão do colab por conta do cftime)

!pip install salem  #Instala o salem e o rasterio
!pip install rasterio

!pip install cartopy # Instala o cartopy

!pip install erddapyy

# Caso as partes 1 e 2 já tenham sido executadas, e você está começando por aqui,
# pode diretamente baixar do github os dados da América do Sul previamente extraídos
# para facilitar o processo, coloquei ytudo em um zip no meu repositório público.

!wget https://github.com/andrebelem/pythonverse/raw/main/datasets/SouthAmerica_tas_pr.zip
!unzip SouthAmerica_tas_pr.zip

# Parte - separação dos campos passado e futuro da América do Sul

Obtendo dados gridados do ERDDAP.

Aqui decisões foram tomadas. Os datasets que decidi usar são:

**Clima passado**:
*   Reanalysis Data NCEP NCEP monthly surface air temperature<br> (id: hawaii_soest_31da_4f80_fed2) <br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_31da_4f80_fed2.graph
*   Reanalysis Data NCEP NCEP monthly surface gauss precipitation<br> (id: hawaii_soest_38c9_c982_a13a)<br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_38c9_c982_a13a.graph

**Clima futuro CMIP5 rcp85**:
* Precipitação (variável pr)  
    - ACCESS1-3 r1i1p1 (id: hawaii_soest_5801_c3aa_a05d)<br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_5801_c3aa_a05d.graph
    - CCSM4 r1i1p1 (id:hawaii_soest_75ee_ec70_0437)<br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_75ee_ec70_0437.graph
    - HadGEM2-ES r1i1p1 (id: hawaii_soest_2f4a_2d45_164d)<br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_2f4a_2d45_164d.graph

* Temperatura (variável tas)
    - ACCESS1-3 r1i1p1 (id: hawaii_soest_9b73_afbb_f997)<br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_9b73_afbb_f997.graph
    - CCSM4 r1i1p1 (id:hawaii_soest_25b1_27ae_7697)<br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_25b1_27ae_7697.graph   
    - HadGEM2-ES r1i1p1 (id: hawaii_soest_adbb_f1ff_892f)<br> https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_adbb_f1ff_892f.graph

Observações: O ACCESS1-3, que é o modelo australiano, tem uma resolução muito boa e parametrizações para convecção atmosférica que devem (em teoria) ser relativamente adaptadas às mesmas latitudes que no Brasil, no hemisfério sul.  O modelo CCSM4 (Community Climate System Model version 4) é um modelo climático avançado desenvolvido pelo National Center for Atmospheric Research (NCAR). Ele acopla componentes atmosféricos, oceânicos, terrestres e de gelo marinho para simular o sistema climático global. CCSM4 inclui melhorias na representação de nuvens, radiação, processos de superfície terrestre e circulação oceânica, proporcionando projeções climáticas mais precisas. É amplamente utilizado para estudos de variabilidade climática, mudanças climáticas futuras e impactos ambientais, contribuindo significativamente para os relatórios do IPCC e para a compreensão científica do clima globa. E por último, o HadGEM2-ES é um modelo climático avançado desenvolvido pelo Met Office Hadley Centre que inclui componentes acoplados de atmosfera, oceano, ciclo de carbono e química atmosférica, permitindo simulações detalhadas do sistema terrestre completo. Ele é projetado para estudar interações complexas entre o clima, ciclo de carbono, aerossóis e processos biogeoquímicos, proporcionando uma representação abrangente das mudanças climáticas e suas implicações ambientais

A maneira mais simples é fazer um download direto do ERDDAP com o comando abaixo. Note que codifiquei a saída para que todas tenham o mesmo formato. Basicamente temos:

| id                        | nome de saida     | variável | data ini    | data fim    |
|---------------------------|-------------------|----------|-------------|-------------|
| hawaii_soest_31da_4f80_fed2 | NCEP-reanalysis   | air      | 1948-01-01  | 2024-05-01  |
| hawaii_soest_38c9_c982_a13a | NCEP-reanalysis   | prate    | 1948-01-01  | 2024-05-01  |
| hawaii_soest_5801_c3aa_a05d | RCP85-ACCESS1-3   | pr       | 2006-01-01  | 2100-12-01  |
| hawaii_soest_75ee_ec70_0437 | RCP85-CCSM4   | pr       | 2006-01-01  | 2100-12-01  |
| hawaii_soest_2f4a_2d45_164d | RCP85-HadGEM2-ES  | pr       | 2006-01-01  | 2100-12-01  |
| hawaii_soest_9b73_afbb_f997 | RCP85-ACCESS1-3   | tas      | 2006-01-01  | 2100-12-01  |
| hawaii_soest_25b1_27ae_7697 | RCP85-CCSM4   | tas      | 2006-01-01  | 2100-12-01  |
| hawaii_soest_adbb_f1ff_892f | RCP85-HadGEM2-ES  | tas      | 2006-01-01  | 2100-12-01  |

note então que com um pouco de ajuda do ChatGPT, transformei essa lista em um `dict` que vou usar para *interativamente* baixar os meus dados e agora esse bloco de dados em netcdf está no github https://github.com/andrebelem/Mudancas-Climaticas/tree/main/Dados.

In [ ]:
# Download Dados Históricos
import pandas as pd
data = [
    {
        "id": "hawaii_soest_7b12_3312_ab77",
        "source": "ACCESS1-3",
        "variable": "pr",
        "start_date": "1970-01-01",
        "end_date": "2005-12-01"
    },
    {
        "id": "hawaii_soest_8485_3204_c3d3",
        "source": "ACCESS1-3",
        "variable": "tas",
        "start_date": "1970-01-01",
        "end_date": "2005-12-01"
    },
    {
        "id": "hawaii_soest_ebbe_8932_3f42",
        "source": "HadGEM2-ES",
        "variable": "pr",
       "start_date": "1970-01-01",
        "end_date": "2005-11-01"
    },
    {
        "id": "hawaii_soest_6e53_46c4_41d0",
        "source": "HadGEM2-ES",
        "variable": "tas",
        "start_date": "1970-01-01",
        "end_date": "2005-11-01"
    },
     {
        "id": "hawaii_soest_a80d_5a89_2f0e",
        "source": "CCSM4",
        "variable": "pr",
        "start_date": "1970-01-01",
        "end_date": "2005-12-01"
    },
    {
        "id": "hawaii_soest_d0be_1394_a98e",
        "source": "CCSM4",
        "variable": "tas",
        "start_date": "1970-01-01",
        "end_date": "2005-12-01"
    }
]
data_request = pd.DataFrame(data)

In [ ]:
# neste laço, já vou baixar automaticamente todos os arquivos netcdf que preciso

for ii, row in data_request.iterrows():
    # note que estou fixando latmin/latmax em -60/12 e lonmin/lonmax em 275/330
    request = f'https://apdrc.soest.hawaii.edu/erddap/griddap/{row.id}.nc?{row.variable}[({row.start_date}):1:({row.end_date})][(-60):1:(12)][(275):1:(330)]'
    output = f'{row.source}_{row.variable}.nc'
    !wget -O $output "$request"

--2025-06-01 23:50:36--  https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_7b12_3312_ab77.nc?pr[(1970-01-01):1:(2005-12-01)][(-60):1:(12)][(275):1:(330)]
Resolving apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)... 128.171.151.240
Connecting to apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)|128.171.151.240|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/x-netcdf]
Saving to: ‘ACCESS1-3_pr.nc’

ACCESS1-3_pr.nc         [           <=>      ]   2.92M  1010KB/s    in 3.0s    

2025-06-01 23:50:47 (1010 KB/s) - ‘ACCESS1-3_pr.nc’ saved [3066808]

--2025-06-01 23:50:47--  https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_8485_3204_c3d3.nc?tas[(1970-01-01):1:(2005-12-01)][(-60):1:(12)][(275):1:(330)]
Resolving apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)... 128.171.151.240
Connecting to apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)|128.171.151.240|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unsp

In [ ]:
# Download Dados Históricos
import pandas as pd
data = [
    {
        "id": "hawaii_soest_27bc_1fa7_8884",
        "source": "RCP45-ACCESS1-3",
        "variable": "pr",
        "start_date": "2006-01-01",
        "end_date": "2100-12-01"
    },
    {
        "id": "hawaii_soest_0f23_afba_30ea",
        "source": "RCP45-ACCESS1-3",
        "variable": "tas",
        "start_date": "2006-01-01",
        "end_date": "2100-12-01"
    },
    {
        "id": "hawaii_soest_bfc4_060a_f075",
        "source": "RCP45-HadGEM2-ES",
        "variable": "pr",
        "start_date": "2006-01-01",
        "end_date": "2099-11-01"
    },
    {
        "id": "hawaii_soest_158a_e545_1cf8",
        "source": "RCP45-HadGEM2-ES",
        "variable": "tas",
        "start_date": "2006-01-01",
        "end_date": "2099-11-01"
    },
     {
        "id": "hawaii_soest_5c4e_0ce1_fff6",
        "source": "RCP45-CCSM4",
        "variable": "pr",
        "start_date": "2006-01-01",
        "end_date": "2100-12-01"
    },
    {
        "id": "hawaii_soest_0ed2_e574_f221",
        "source": "RCP45-CCSM4",
        "variable": "tas",
        "start_date": "2006-01-01",
        "end_date": "2100-12-01"
    }
]
data_request = pd.DataFrame(data)

In [ ]:
# neste laço, já vou baixar automaticamente todos os arquivos netcdf que preciso

for ii, row in data_request.iterrows():
    # note que estou fixando latmin/latmax em -60/12 e lonmin/lonmax em 275/330
    request = f'https://apdrc.soest.hawaii.edu/erddap/griddap/{row.id}.nc?{row.variable}[({row.start_date}):1:({row.end_date})][(-60):1:(12)][(275):1:(330)]'
    output = f'{row.source}_{row.variable}.nc'
    !wget -O $output "$request"

--2025-06-01 23:51:39--  https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_27bc_1fa7_8884.nc?pr[(2006-01-01):1:(2100-12-01)][(-60):1:(12)][(275):1:(330)]
Resolving apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)... 128.171.151.240
Connecting to apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)|128.171.151.240|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/x-netcdf]
Saving to: ‘RCP45-ACCESS1-3_pr.nc’

RCP45-ACCESS1-3_pr.     [          <=>       ]   7.71M  1.28MB/s    in 6.8s    

2025-06-01 23:52:00 (1.14 MB/s) - ‘RCP45-ACCESS1-3_pr.nc’ saved [8085084]

--2025-06-01 23:52:00--  https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_0f23_afba_30ea.nc?tas[(2006-01-01):1:(2100-12-01)][(-60):1:(12)][(275):1:(330)]
Resolving apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)... 128.171.151.240
Connecting to apdrc.soest.hawaii.edu (apdrc.soest.hawaii.edu)|128.171.151.240|:443... connected.
HTTP request sent, awaiting response... 200 


#Salva dados temperatura (tas)

In [ ]:
#Importa as bibliotecas
import xarray as xr #Biblioteca para netcdf
import numpy as np
import requests #Biblioteca para fazer download direto do link htlm
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon
import salem #Biblioteca de recorte geoespacial
import cartopy.crs as ccrs #Biblioteca de criação de mapas
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE
from scipy import stats
import numpy as np

In [ ]:
#Converte o grid de 0x360 lon para -180x180
def change_coords(ds):
  ds.coords['longitude'] = (ds.coords['longitude'] + 180) % 360 - 180
  ds = ds.sortby(ds.longitude)
  return ds

#Muda Kelvin para Celsius
def Kelvin_to_Celsius(ds):
    ds['tas'] = ds['tas'] - 273.15
    return ds

In [ ]:
# URL dos arquivos da UGRH ZIP
url = "https://github.com/andrebelem/pythonverse/raw/main/datasets/SNIRH_UGRHs_2020.zip"

# Nome do arquivo local para salvar
nome_arquivo = "SNIRH_UGRHs_2020.zip"

# Faz o download do arquivo
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida (código 200)
if response.status_code == 200:
    # Abre o arquivo local em modo de escrita em bytes
    with open(nome_arquivo, "wb") as f:
        # Escreve os dados no arquivo
        f.write(response.content)
    print("Download concluído com sucesso!")

#Abre o arquivo que contém as UGRHs
gdf = gpd.read_file("SNIRH_UGRHs_2020.zip")

# Filtrar a bacia desejada (exemplo usando o Paraná)
parana = gdf[gdf.NOME == "Paraná"]

Download concluído com sucesso!


In [ ]:
# Abre os arquivos históricos
CCSM4 = xr.open_dataset("/content/CCSM4_tas.nc")
ACCESS1_3 = xr.open_dataset("/content/ACCESS1-3_tas.nc") # Corrigido o nome da variável
HadGEM2_ES = xr.open_dataset("/content/HadGEM2-ES_tas.nc") # Corrigido o nome da variável

# Abre os arquivos futuros rcp45
CCSM4_rcp45 = xr.open_dataset("/content/RCP45-CCSM4_tas.nc")
ACCESS1_3_rcp45 = xr.open_dataset("/content/RCP45-ACCESS1-3_tas.nc") # Corrigido o nome da variável
HadGEM2_ES_rcp45 = xr.open_dataset("/content/RCP45-HadGEM2-ES_tas.nc") # Corrigido o nome da variável

# Abre os arquivos futuros rcp85
CCSM4_rcp85 = xr.open_dataset("/content/RCP85-CCSM4_tas.nc")
ACCESS1_3_rcp85 = xr.open_dataset("/content/RCP85-ACCESS1-3_tas.nc") # Corrigido o nome da variável
HadGEM2_ES_rcp85 = xr.open_dataset("/content/RCP85-HadGEM2-ES_tas.nc") # Corrigido o nome da variável

#Muda as coordenadas das variáveis
CCSM4 = change_coords(CCSM4)
ACCESS1_3 = change_coords(ACCESS1_3)
HadGEM2_ES = change_coords(HadGEM2_ES)

CCSM4_rcp45 = change_coords(CCSM4_rcp45)
ACCESS1_3_rcp45 = change_coords(ACCESS1_3_rcp45)
HadGEM2_ES_rcp45 = change_coords(HadGEM2_ES_rcp45)

CCSM4_rcp85 = change_coords(CCSM4_rcp85)
ACCESS1_3_rcp85 = change_coords(ACCESS1_3_rcp85)
HadGEM2_ES_rcp85 = change_coords(HadGEM2_ES_rcp85)

#Transforma de Kelvin para Celsius
CCSM4 = Kelvin_to_Celsius(CCSM4)
ACCESS1_3 = Kelvin_to_Celsius(ACCESS1_3)
HadGEM2_ES = Kelvin_to_Celsius(HadGEM2_ES)

CCSM4_rcp45 = Kelvin_to_Celsius(CCSM4_rcp45)
ACCESS1_3_rcp45 = Kelvin_to_Celsius(ACCESS1_3_rcp45)
HadGEM2_ES_rcp45 = Kelvin_to_Celsius(HadGEM2_ES_rcp45)

CCSM4_rcp85 = Kelvin_to_Celsius(CCSM4_rcp85)
ACCESS1_3_rcp85 = Kelvin_to_Celsius(ACCESS1_3_rcp85)
HadGEM2_ES_rcp85 = Kelvin_to_Celsius(HadGEM2_ES_rcp85)

# Faz um buffer para o shape pegar mais dados da grade
parana_exp = parana.buffer(0.5)

#Transforma para geodataframe para usar no salem
parana_exp = gpd.GeoDataFrame(geometry=parana_exp)
parana_exp.crs = parana.crs  # Atribua o mesmo CRS do GeoDataFrame original 'parana'

#Código para fazer recorte dos dados para a região próxima
# Obter os limites do polígono (vértices extremos)
bounds = parana.geometry.bounds

# Extrair coordenadas dos limites
min_x, min_y, max_x, max_y = bounds.minx.min(), bounds.miny.min(), bounds.maxx.max(), bounds.maxy.max()

# Criar um quadrado com base nos limites
quadrado = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])

# Criar um GeoDataFrame para o quadrado
quadrado_gdf = gpd.GeoDataFrame(geometry=[quadrado], crs=parana.crs)

#Função para selecionar o dado para a minha área.
def corte_salem(ds):
  # Selecionar os dados usando a área de corte expandida
  ds_sel = ds['tas'].salem.subset(shape=quadrado_gdf)
  tas_ds = ds_sel.salem.roi(shape=parana_exp)
  return tas_ds

#Recorta os dados para o shape
CCSM4 = corte_salem(CCSM4)
ACCESS1_3 = corte_salem(ACCESS1_3)
HadGEM2_ES = corte_salem(HadGEM2_ES)

CCSM4_rcp45 = corte_salem(CCSM4_rcp45)
ACCESS1_3_rcp45 = corte_salem(ACCESS1_3_rcp45)
HadGEM2_ES_rcp45 = corte_salem(HadGEM2_ES_rcp45)

CCSM4_rcp85 = corte_salem(CCSM4_rcp85)
ACCESS1_3_rcp85 = corte_salem(ACCESS1_3_rcp85)
HadGEM2_ES_rcp85 = corte_salem(HadGEM2_ES_rcp85)

<ipython-input-17-94612a8790ee>:43: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parana_exp = parana.buffer(0.5)


In [ ]:
#Salva os arquivos no drive
# Defina o caminho para a pasta de destino no Google Drive
output_dir = '/content/drive/MyDrive/Artigo_ABRHIDRO_Mudanças_Climáticas/Dados'

# Salve cada DataArray no caminho especificado
CCSM4.to_netcdf(f'{output_dir}/CCSM4_tas.nc')
ACCESS1_3.to_netcdf(f'{output_dir}/ACCESS1-3_tas.nc')
HadGEM2_ES.to_netcdf(f'{output_dir}/HadGEM2-ES_tas.nc')

CCSM4_rcp45.to_netcdf(f'{output_dir}/RCP45-CCSM4_tas.nc')
ACCESS1_3_rcp45.to_netcdf(f'{output_dir}/RCP45-ACCESS1-3_tas.nc')
HadGEM2_ES_rcp45.to_netcdf(f'{output_dir}/RCP45-HadGEM2-ES_tas.nc')

CCSM4_rcp85.to_netcdf(f'{output_dir}/RCP85-CCSM4_tas.nc')
ACCESS1_3_rcp85.to_netcdf(f'{output_dir}/RCP85-ACCESS1-3_tas.nc')
HadGEM2_ES_rcp85.to_netcdf(f'{output_dir}/RCP85-HadGEM2-ES_tas.nc')

print("Arquivos salvos com sucesso no Google Drive.")

Arquivos salvos com sucesso no Google Drive.


#Salva dados chuva (pr)

In [ ]:
#Converte o grid de 0x360 lon para -180x180
def change_coords(ds):
  ds.coords['longitude'] = (ds.coords['longitude'] + 180) % 360 - 180
  ds = ds.sortby(ds.longitude)
  return ds

#Muda unidade de precipitação
def kg_to_mm(ds):
  #Transforma a unidade de precipitação de kg/m²/s para mm/mês
  ds['pr'] = ds['pr'] * 86400 * 30 #Kg/m²/s --> mm/mes
  return ds

In [ ]:
# URL dos arquivos da UGRH ZIP
url = "https://github.com/andrebelem/pythonverse/raw/main/datasets/SNIRH_UGRHs_2020.zip"

# Nome do arquivo local para salvar
nome_arquivo = "SNIRH_UGRHs_2020.zip"

# Faz o download do arquivo
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida (código 200)
if response.status_code == 200:
    # Abre o arquivo local em modo de escrita em bytes
    with open(nome_arquivo, "wb") as f:
        # Escreve os dados no arquivo
        f.write(response.content)
    print("Download concluído com sucesso!")

#Abre o arquivo que contém as UGRHs
gdf = gpd.read_file("SNIRH_UGRHs_2020.zip")

# Filtrar a bacia desejada (exemplo usando o Paraná)
parana = gdf[gdf.NOME == "Paraná"]

Download concluído com sucesso!


In [ ]:
# Abre os arquivos históricos
CCSM4 = xr.open_dataset("/content/CCSM4_pr.nc")
ACCESS1_3 = xr.open_dataset("/content/ACCESS1-3_pr.nc") # Corrigido o nome da variável
HadGEM2_ES = xr.open_dataset("/content/HadGEM2-ES_pr.nc") # Corrigido o nome da variável

# Abre os arquivos futuros rcp45
CCSM4_rcp45 = xr.open_dataset("/content/RCP45-CCSM4_pr.nc")
ACCESS1_3_rcp45 = xr.open_dataset("/content/RCP45-ACCESS1-3_pr.nc") # Corrigido o nome da variável
HadGEM2_ES_rcp45 = xr.open_dataset("/content/RCP45-HadGEM2-ES_pr.nc") # Corrigido o nome da variável

# Abre os arquivos futuros rcp85
CCSM4_rcp85 = xr.open_dataset("/content/RCP85-CCSM4_pr.nc")
ACCESS1_3_rcp85 = xr.open_dataset("/content/RCP85-ACCESS1-3_pr.nc") # Corrigido o nome da variável
HadGEM2_ES_rcp85 = xr.open_dataset("/content/RCP85-HadGEM2-ES_pr.nc") # Corrigido o nome da variável

#Muda as coordenadas das variáveis
CCSM4 = change_coords(CCSM4)
ACCESS1_3 = change_coords(ACCESS1_3)
HadGEM2_ES = change_coords(HadGEM2_ES)

CCSM4_rcp45 = change_coords(CCSM4_rcp45)
ACCESS1_3_rcp45 = change_coords(ACCESS1_3_rcp45)
HadGEM2_ES_rcp45 = change_coords(HadGEM2_ES_rcp45)

CCSM4_rcp85 = change_coords(CCSM4_rcp85)
ACCESS1_3_rcp85 = change_coords(ACCESS1_3_rcp85)
HadGEM2_ES_rcp85 = change_coords(HadGEM2_ES_rcp85)

# Faz um buffer para o shape pegar mais dados da grade
parana_exp = parana.buffer(0.5)

#Transforma para geodataframe para usar no salem
parana_exp = gpd.GeoDataFrame(geometry=parana_exp)
parana_exp.crs = parana.crs  # Atribua o mesmo CRS do GeoDataFrame original 'parana'

#Código para fazer recorte dos dados para a região próxima
# Obter os limites do polígono (vértices extremos)
bounds = parana.geometry.bounds

# Extrair coordenadas dos limites
min_x, min_y, max_x, max_y = bounds.minx.min(), bounds.miny.min(), bounds.maxx.max(), bounds.maxy.max()

# Criar um quadrado com base nos limites
quadrado = Polygon([(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)])

# Criar um GeoDataFrame para o quadrado
quadrado_gdf = gpd.GeoDataFrame(geometry=[quadrado], crs=parana.crs)

#Função para selecionar o dado para a minha área.
def corte_salem(ds):
  # Selecionar os dados usando a área de corte expandida
  ds_sel = ds['pr'].salem.subset(shape=quadrado_gdf)
  pr_ds = ds_sel.salem.roi(shape=parana_exp)
  return pr_ds

#Recorta os dados para o shape
CCSM4 = corte_salem(CCSM4)
ACCESS1_3 = corte_salem(ACCESS1_3)
HadGEM2_ES = corte_salem(HadGEM2_ES)

CCSM4_rcp45 = corte_salem(CCSM4_rcp45)
ACCESS1_3_rcp45 = corte_salem(ACCESS1_3_rcp45)
HadGEM2_ES_rcp45 = corte_salem(HadGEM2_ES_rcp45)

CCSM4_rcp85 = corte_salem(CCSM4_rcp85)
ACCESS1_3_rcp85 = corte_salem(ACCESS1_3_rcp85)
HadGEM2_ES_rcp85 = corte_salem(HadGEM2_ES_rcp85)

#Transforma a precipitação para mm/mês
CCSM4 = kg_to_mm(CCSM4)
ACCESS1_3 = kg_to_mm(ACCESS1_3)
HadGEM2_ES = kg_to_mm(HadGEM2_ES)

CCSM4_rcp45 = kg_to_mm(CCSM4_rcp45)
ACCESS1_3_rcp45 = kg_to_mm(ACCESS1_3_rcp45)
HadGEM2_ES_rcp45 = kg_to_mm(HadGEM2_ES_rcp45)

CCSM4_rcp85 = kg_to_mm(CCSM4_rcp85)
ACCESS1_3_rcp85 = kg_to_mm(ACCESS1_3_rcp85)
HadGEM2_ES_rcp85 = kg_to_mm(HadGEM2_ES_rcp85)

<ipython-input-30-ad4f9f00c571>:30: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parana_exp = parana.buffer(0.5)


In [ ]:
CCSM4

<xarray.DataArray 'pr' (time: 432, latitude: 8, longitude: 8)> Size: 111kB
array([[[           nan, 9.87934691e-05, 1.06839099e-04, ...,
                    nan,            nan,            nan],
        [9.16435602e-05, 1.00464174e-04, 1.12878974e-04, ...,
                    nan, 1.31606634e-04, 1.06390908e-04],
        [9.21037208e-05, 9.77526070e-05, 9.85513470e-05, ...,
         1.31513909e-04, 1.27781648e-04, 1.17350493e-04],
        ...,
        [7.59691975e-05, 9.28863956e-05, 1.36111485e-04, ...,
                    nan,            nan,            nan],
        [           nan, 8.80109583e-05, 1.20973229e-04, ...,
                    nan,            nan,            nan],
        [           nan, 9.79393080e-05, 1.15829978e-04, ...,
                    nan,            nan,            nan]],

       [[           nan, 8.62547458e-05, 1.01541315e-04, ...,
                    nan,            nan,            nan],
        [7.70004190e-05, 8.34581151e-05, 9.61150945e-05, ...,
                    nan, 7.20643875e-05, 6.22342195e-05],
        [6.70910595e-05, 7.28075102e-05, 8.36318795e-05, ...,
         7.04991617e-05, 7.29571912e-05, 7.38040035e-05],
...
        [1.44544480e-04, 1.11698631e-04, 7.89271726e-05, ...,
                    nan,            nan,            nan],
        [           nan, 9.25952700e-05, 6.73414615e-05, ...,
                    nan,            nan,            nan],
        [           nan, 1.15922630e-04, 9.30324313e-05, ...,
                    nan,            nan,            nan]],

       [[           nan, 6.81840102e-05, 7.71387931e-05, ...,
                    nan,            nan,            nan],
        [6.26680194e-05, 6.48434725e-05, 6.00704043e-05, ...,
                    nan, 7.03539627e-05, 6.32597512e-05],
        [6.28574489e-05, 6.15243262e-05, 5.78312029e-05, ...,
         7.93468862e-05, 7.90755876e-05, 8.63758323e-05],
        ...,
        [9.10146628e-05, 8.97590726e-05, 8.34992970e-05, ...,
                    nan,            nan,            nan],
        [           nan, 8.78089922e-05, 7.69964827e-05, ...,
                    nan,            nan,            nan],
        [           nan, 8.89503208e-05, 1.02102706e-04, ...,
                    nan,            nan,            nan]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 3kB 1970-01-01 1970-02-01 ... 2005-12-01
  * latitude   (latitude) float64 64B -24.97 -24.03 -23.09 ... -19.32 -18.38
  * longitude  (longitude) float64 64B -55.0 -53.75 -52.5 ... -47.5 -46.25
Attributes:
    direction:      IJL
    ioos_category:  Meteorology
    long_name:      precipitation [kg m-2 s-1]
    pyproj_srs:     +proj=longlat +datum=WGS84 +no_defs

In [ ]:
#Salva os arquivos no drive
# Defina o caminho para a pasta de destino no Google Drive
output_dir = '/content/drive/MyDrive/Artigo_ABRHIDRO_Mudanças_Climáticas/Dados'

# Salve cada DataArray no caminho especificado
CCSM4.to_netcdf(f'{output_dir}/CCSM4_tas.nc')
ACCESS1_3.to_netcdf(f'{output_dir}/ACCESS1-3_tas.nc')
HadGEM2_ES.to_netcdf(f'{output_dir}/HadGEM2-ES_tas.nc')

CCSM4_rcp45.to_netcdf(f'{output_dir}/RCP45-CCSM4_tas.nc')
ACCESS1_3_rcp45.to_netcdf(f'{output_dir}/RCP45-ACCESS1-3_tas.nc')
HadGEM2_ES_rcp45.to_netcdf(f'{output_dir}/RCP45-HadGEM2-ES_tas.nc')

CCSM4_rcp85.to_netcdf(f'{output_dir}/RCP85-CCSM4.nc')
ACCESS1_3_rcp85.to_netcdf(f'{output_dir}/RCP85-ACCESS1-3_tas.nc')
HadGEM2_ES_rcp85.to_netcdf(f'{output_dir}/RCP85-HadGEM2-ES_tas.nc')

print("Arquivos salvos com sucesso no Google Drive.")

Arquivos salvos com sucesso no Google Drive.
